# Gene Sequences

In [4]:
Set_Strings = [(0, 'CAGCGGGTGCGTAATTTGGAGAAGTTATTCTGCAACGAAATCAATCCTGTTTCGTTAGCTTACGGACTACGACGAGAGGGTACTTCCCTGATATAGTCAC'),
(1, 'CAAGTCGGGCGTATTGGAGAATATTTAAATCGGAAGATCATGTTACTATGCGTTAGCTCACGGACTGAAGAGGATTCTCTCTTAATGCAA'),
(2, 'CATGGGTGCGTCGATTTTGGCAGTAAAGTGGAATCGTCAGATATCAATCCTGTTTCGTAGAAAGGAGCTACCTAGAGAGGATTACTCTCACATAGTA'),
(3, 'CAAGTCCGCGATAAATTGGAATATTTGTCAATCGGAATAGTCAACTTAGCTGGCGTTAGCTTTACGACTGACAGAGAGAAACCTGTCCATCACACA'),
(4, 'CAGTCCGGCGTAATTGGAGAATATTTTGCAATCGGAAGATCAATCTTGTTAGCGTTAGCTTACGACTGACGAGAGGGATACTCTCTCTAATACAA'),
(5, 'CACGGGCTCCGCATCTATTTTGGGTCAAGTTGCATATCAGTCATCGACAATCAAACACTGTTTTGCGGTAGATAAGATACGACTGAGAGAGGACGTTCGCTCGAATATAGTTAC'),
(6, 'CACGGGTCCAATTTTGGAGTAAGTTGATATCGTCACGAAATCAATCCTGTTTCGGTAGTATAGGACTACGACGAGAGAGGACGTTCCTCTGATATAGTTAC'), 
(7, 'GGTCCGTCAATTTTGGAGTAAGTTGATATCGTCACGAAATCAATCCTGTTTCGGTAGTATAGGACTACGACGAGAGAGGACGTTCCTCTGATATAGTTAC'), 
(8, 'CACGGGAATCCGTCAATTTTGGAGTAAGTTGATATCGTCACGAAATCAATCCTGTTTCGGTAGTATAGGACTACGACGAGAGAGGACGTTCCTCTGATATAGTTAC'), 
(9, 'CACGGGTCCGTCAATTTTGGAGTAAGTTGATATCGTCACGAAATCAATCCTGTTTCGGTAGTATAGGACTACGACGAGAGAGGACGTTCCTCTGATATAGTTAC')]

# 1. LCS 

In [5]:
# bottom up dynamic programming
def lcs(X, Y): 
    m = len(X) 
    n = len(Y) 

    # Store LCS between X ending in m and Y ending in n
    L = [[None]*(n + 1) for _ in range(m + 1)] 
  
    for i in range(m + 1): 
        for j in range(n + 1): 
            # if one of the strings is null, LCS is 0
            if i == 0 or j == 0 : 
                L[i][j] = 0
            # if last elem match, LCS += 1
            elif X[i-1] == Y[j-1]: 
                L[i][j] = L[i-1][j-1]+1
            # else take LCS of removing last elem from X or from Y
            else: 
                L[i][j] = max(L[i-1][j], L[i][j-1]) 
    return L

# 2. LCS for each Gene Pair

In [100]:
# generate table of LCS 
def similarityTable(strings):
    sim_table = [[None]*(len(strings)) for _ in range(len(strings))]

    # calculate LCS for all pairs
    for i in range(len(strings)):
        for j in range(len(strings)):
            # don't make repeated comparisons
            if j < i:
                sim_table[i][j] = sim_table[j][i]
            # if comparing string with itself, leave LCS as 0
            elif i!=j:
                # store LCS by taking last entry in store
                sim_table[i][j] = lcs(strings[i][1],strings[j][1])[-1][-1]
    return sim_table

In [101]:
t = similarityTable(Set_Strings)
print(t)

[[None, 74, 76, 73, 82, 84, 89, 87, 91, 91], [74, None, 67, 72, 79, 71, 69, 68, 71, 71], [76, 67, None, 65, 69, 82, 82, 81, 84, 84], [73, 72, 65, None, 80, 72, 68, 67, 69, 69], [82, 79, 69, 80, None, 74, 74, 73, 75, 75], [84, 71, 82, 72, 74, None, 95, 93, 97, 97], [89, 69, 82, 68, 74, 95, None, 97, 101, 101], [87, 68, 81, 67, 73, 93, 97, None, 100, 100], [91, 71, 84, 69, 75, 97, 101, 100, None, 104], [91, 71, 84, 69, 75, 97, 101, 100, 104, None]]


In [103]:
def sim_w_family(sim_table):
    return([sum(filter(None,x)) for x in t])

print(sim_w_family(t))

[747, 642, 690, 635, 681, 765, 776, 766, 792, 792]


# 3. Manually infer relationships

str8 is likely the first generation (origin), since it has the most similarity to all other strings, indicating it is high up the geneology tree.

str9 and str6 are likely the second generation since they have the highest LCS with str8.

str6's children are likely str5 and str7 since they have the highest LCS with str6 from the unassigned strings.

str9's children are likely str0 and str2 since they have the highest LCS with str9 from the unassigned strings.

str0 is likely the parent of str1 and str4 as they have very high LCS with str0

str2 is likely the parent of str3 since it has the highest LCS amongst remaining current "leaf" strings.

                                  str8
                     str6                         str9
          str5                str7        str2                str0
                                      str3                str1   str4

# 4. Geneology Algo

In [167]:
# tree node class
class Node():
    def __init__(self,val):
        self.val = val
        self.child = [None,None]
    def __str__(self):
        return(str(self.val))

# helper func
def _clear_col(table,col):
    for row in range(len(table)):
        table[row][col] = None
    return(table) 
        
def create_tree(genes):
    # get LCS for each pair
    t = similarityTable(genes)
    
    # get general similarity to all other strings
    conf = sim_w_family(t)

    # assume the one with the most LCS with 
    # all the other strings is the origin
    root = Node(conf.index(max(conf)))
    # clear column of assigned gene
    _clear_col(t,root.val)
    
    # create tree in a BFS manner
    q = [root]
    
    # while queue is not empty
    #while q:
    while q:
        c = q.pop(0)
        # form geneology tree by greedily taking max LCS from root
        c.child = [Node(t[c.val].index(x)) for x in sorted(filter(None,t[c.val]))[-2:]]
        
        # add assigned children to queue
        q.extend(c.child)
        
        # clear cols of assigned genes and pad node children
        # if no children
        if not c.child:
            c.child = [None,None]
        
        # if one child
        elif len(c.child) < 2:
            c.child.append(None)
            _clear_col(t,c.child[0].val)
        
        # if two child
        else:
            _clear_col(t,c.child[0].val)
            _clear_col(t,c.child[1].val)
    return(root)

In [152]:
r = create_tree(Set_Strings)
a1, a2 = r.child

b1, b2 = a1.child
b3, b4 = a2.child

c1,c2 = b1.child
c3,_ = b2.child

In [153]:
print(r)
print(a1,a2)
print(b1,b2,b3,b4)
print(c1,c2,c3)

8
6 9
5 7 2 0
3 4 1


_____

# 5. Strengths and Weaknesses

**Strengths**      
Simple: It is a very simple and intuitive algorithm, greedily determining the root and the next children by the highest LCS. This makes it easy to maintain.

Time Efficiency: As explained in part 5, LCS takes M^2. sunce we do LCS for every pair constructing the LCS table takes N^2(M^2) time. After getting the LCS table, we assign children for each node (N), the assignment of children itself takes around (lgN) time due to the sort, overall assigning children for all nodes (constructing the tree) takes O(NlgN). Overall the algorithm takes $N^2M^2$ + $NlgN$ time, which is O($N^2M^2$) time.               

Space Efficiency: It takes O(n^2) space (storing the LCS for each pair) which is decent.


**Weaknesses**      
Does not guarantee global optimumz: since it takes the greedy approach, it is possible that by taking the locally optimum solution it is rejecting the globally optimal solution (especially since selected children now eliminate them from being some other node's children, even if it has a longer LCS with that future node).

Assumes specific family tree structure: The algorithm assumes a maximally balanced/ full tree, determing the best two children at each time if possible. This is particularly a problem in the last generation when only some of the current leaf nodes will have children. The algorithm will not try to determine which of the leaf nodes will most likely have children, it will just assign children greedily in sequential order. 

# 6. Computational Complexity

O($N^2M^2$)

As explained in part 5, LCS takes M^2. sunce we do LCS for every pair constructing the LCS table takes N^2(M^2) time. After getting the LCS table, we assign children for each node (N), the assignment of children itself takes around (lgN) time due to the sort, overall assigning children for all nodes (constructing the tree) takes O(NlgN). Overall the algorithm takes $N^2M^2$ + $NlgN$ time, which is O($N^2M^2$) time.

# 7. Probabilities of insertions, deletions, and changes

Go through the family tree created and count the number of insertions, deletions, along each parent to child edge (assuming the combination that minimizes the total number of mutations) as well as the number of genes that had the possibility of mutating. Then return the percentage of actual insertions/ deletions/ changes relative to total characters that could have mutated.

In [154]:
# calculate similarities
def editDist(s1, s2): 
    m = len(s1)
    n = len(s2)
    # store subsolutions
    dp = [[0]*(n+1) for _ in range(m+1)] 
  
    # for every s1 ending in m and s2 ending in n
    for i in range(m+1): 
        for j in range(n+1): 
  
            # if a str is empty, insert all chars of other str
            if i == 0 or j ==0: 
                dp[i][j] = max(i,j)
  
            # if last char same sol is just edit dist
            # of the strings without last char
            elif s1[i-1] == s2[j-1]: 
                dp[i][j] = dp[i-1][j-1] 
  
            # else, sol is minimum dp
            # from all possible operations
            else: 
                dp[i][j] = 1 + min(dp[i][j-1],      # insert 
                                   dp[i-1][j],      # remove 
                                   dp[i-1][j-1])    # change 
  
    return dp 

In [173]:
def count_mutations(gene1, gene2):
    store = editDist(gene1, gene2)
    ins_c = 0
    del_c = 0
    cha_c = 0
    m = len(store) - 1
    n = len(store[0]) - 1
    while m != 0 and n != 0:
        if store[m][n] == store[m-1][n-1]:
            # equal, no edits
            m -= 1
            n -= 1
        else:
            min_ed = min(store[m][n-1],store[m-1][n],store[m-1][n-1])
            if store[m][n-1] == min_ed:
                ins_c += 1
                n -= 1
            elif store[m-1][n] == min_ed:
                del_c += 1
                m -= 1
            else:
                cha_c += 1
                m -= 1
                n -= 1
    return(ins_c,del_c,cha_c)

def count_mutations_tree(root):
    ins_total = 0
    del_total = 0
    cha_total = 0
    char_total = 0
    q = [root]
    
    # while there are still nodes to check
    while q:
        c = q.pop(0)
        char_total += len(Set_Strings[c.val][1])
        
        # for all children
        for ch in filter(None,c.child):
            # record mutations
            ins_c,del_c,cha_c = count_mutations(Set_Strings[c.val][1], Set_Strings[ch.val][1])
            ins_total += ins_c
            del_total += del_c
            cha_total += cha_c
            
            # add children to q
            q.append(ch)
    return(ins_total, del_total,cha_total,char_total)

ins_total, del_total, chat_total,char_total = count_mutations_tree(r)

print('Probability of Mutations')
print('Insertions: ', round(ins_total/char_total,2))
print('Deletions: ', round(del_total/char_total,2))
print('Changes: ', round(cha_total/char_total,2))

Probability of Mutations
Insertions:  0.05
Deletions:  0.09
Changes:  0.48


# 8. HCs

**#composition: ***       
I used classes when it made sense and simplified my code, and divided my code into separate modules. Overall this helps the code be more readable as there are chunks of functionality that build on each other.

**#decisionselection: ***       
Out of all the manual approaches I tried, (finding clusters then putting it together, trying brute force, constructing best subtrees and building out, etc) the most effective approach was just to successively approximate (make a best guess for the children of the current node, and then do it for the nodes just added) which is why I ended up doing a greedy program.

______